# Feasibility of the Youtube Comedy Slam Project

## Introduction

Initial experiments and exploration of the data showed some promise for being able to decide which video of 2 is funnier. The exploration can be found [here](https://github.com/papapana/data_science/blob/master/youtube_comedy_slam/notebooks/Youtube%20Data%20Exploration.ipynb). The dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/YouTube+Comedy+Slam+Preference+Data). 

The dataset contains information about which video is funnier for many pairs of videos. In order to builld a general model that can distinguish between any video which one is funnier we create a score for each video based on our information and later we try to model the score so that videos can be compared.

In this notebook we show how to build such a score using multiple ways, and show how all these fail to get an acceptable score



## Loading the datasets

In [31]:
%matplotlib inline
import matplotlib.pyplot as plt
import gensim
import pandas as pd
import pickle
import statsmodels.formula.api as st
import numpy as np
import networkx as nx
from scipy import io
from sklearn import linear_model
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score, mean_squared_error

We have training and test sets given by the dataset itself.
The video_score_train is for each video `#funnier_than_other_videos - #not_funnier_than_other_videos`

In [12]:
initial_ds_train = pd.read_csv('../data/processed/comedy_comparisons.train')
initial_ds_test = pd.read_csv('../data/processed/comedy_comparisons.test')
initial_ds_train.funnier[initial_ds_train.funnier == 'left'] = 0
initial_ds_train.funnier[initial_ds_train.funnier == 'right'] = 1
initial_ds_test.funnier[initial_ds_test.funnier == 'left'] = 0
initial_ds_test.funnier[initial_ds_test.funnier == 'right'] = 1
video_score_train = pickle.load(open('../data/processed/video_score_train.p', 'rb'))
video_score_test = pickle.load(open('../data/processed/video_score_test.p', 'rb'))
initial_ds_train.head()


,video_id1,video_id2,funnier
0,sNabaB-eb3Y,wHkPb68dxEw,0
1,sNabaB-eb3Y,y2emSXSE-N4,0
2,Vr4D8xO2lBY,sNabaB-eb3Y,1
3,sNabaB-eb3Y,dDtRnstrefE,0
4,vX95JgKGu0o,wu4SU70w7LA,0


In [32]:
def predict_initial_problem(initial_dataset, video_score_test):
    bin_ytest = []
    for row in initial_dataset.itertuples():
        pred1 = video_score_test[row.video_id1] if row.video_id1 in video_score_test else 0.0
        pred2 = video_score_test[row.video_id2] if row.video_id2 in video_score_test else 0.0
        bin_ytest += [0 if pred1 >= pred2 else 1]
    return bin_ytest

y_pred_train = predict_initial_problem(initial_ds_train, video_score_train)
y_pred_test = predict_initial_problem(initial_ds_test, video_score_test)



In [33]:
y_train = list(initial_ds_train.funnier.tolist())
y_test = list(initial_ds_test.funnier.tolist())

print(metrics.classification_report(y_test, y_pred_test))
print("Confusion Matrix:")
print(metrics.confusion_matrix(y_test, y_pred_test))
print("Accuracy:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred_test))

             precision    recall  f1-score   support

          0       0.48      0.65      0.55     18884
          1       0.49      0.32      0.39     19713

avg / total       0.48      0.48      0.47     38597

Confusion Matrix:
[[12361  6523]
 [13439  6274]]
Accuracy: 0.482809544783


We see that the accuracy of the first scoring system is only 48% which is worse than chance!


## Modeling the problem as a graph

In this section we model the dataset as a directed graph where the nodes are videos and the edges are directed to the funnier of the 2 videos.

We use the library networkx to help us.

In [35]:
def get_ds_as_graph(ds):
    G = nx.DiGraph()
    for row in ds.itertuples():
        G.add_edges_from([(row.video_id1, row.video_id2)] if row.funnier else [(row.video_id2, row.video_id1)])
    return G

In [36]:
g_train = get_ds_as_graph(initial_ds_train)
g_test = get_ds_as_graph(initial_ds_test)

## Properties of the graph and their meaning

First of all, we want to check if in the dataset the transitive property holds.
That is if video A is funnier than video B and B is funnier than C, then A is funnier than C.
In our graph this would mean that there is an edge $A\leftarrow B$ and $B\leftarrow C$  but not $C\leftarrow A$. This would create a cycle and break the transitive property.

So, we want the graph to be a directed acyclic graph (DAG) so that we can use comparisons in a consistent way.

We test both the train and test graph

In [37]:
nx.is_directed_acyclic_graph(g_train)

False

In [38]:
nx.is_directed_acyclic_graph(g_test)

False

We see that they are not DAGs which is bad for creating a score where we are sure about the hierarchy of videos.

Then we proceed to check if we have all the relations we need i.e. every video is related to every other through some intermediate videos e.g. if we want to see if video A is funnier than Z then there is some connection `path` from A to Z. This would be translated in the video graph if it is strongly connected.

In [39]:
nx.is_strongly_connected(g_train)

False

In [40]:
nx.is_strongly_connected(g_test)

False

The graphs are not strongly connected and therefore we cannot make claims for every pair of videos.

We now check if at least there is some connection of the videos without taking into account direction (which one is funnier).

In [41]:
nx.number_weakly_connected_components(g_train)

74

In [42]:
nx.number_weakly_connected_components(g_test)

7

Since the number of weakly connected components is not 1 there is no connection between every 2 videos

We now check if we score the importance of each node using this [network clustering algorithm](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.cluster.clustering.html#networkx.algorithms.cluster.clustering) if we get more predictive power

In [27]:
g_test_undirected = g_test.to_undirected()
video_score_test_clustered = nx.clustering(g_test_undirected)
y_pred_test_clustered = predict_initial_problem(initial_ds_test, video_score_test_clustered)
print(metrics.classification_report(y_test, y_pred_test_clustered))
print("Confusion Matrix:")
print(metrics.confusion_matrix(y_test, y_pred_test_clustered))
print("Accuracy:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred_test_clustered))

             precision    recall  f1-score   support

          0       0.49      0.54      0.51     18884
          1       0.51      0.47      0.49     19713

avg / total       0.50      0.50      0.50     38597

Confusion Matrix:
[[10120  8764]
 [10487  9226]]
Accuracy: 0.501230665596


Still we are at 50% and therefore this is not a good scoring system either for this dataset

Now we try the [pagerank algorithm](https://en.wikipedia.org/wiki/PageRank) that is a good algorithm for assigning weights to nodes of a directed graph.

In [43]:
pr = nx.pagerank(g_test, alpha=0.9)
y_pred_test_pagerank = predict_initial_problem(initial_ds_test, pr)
print(metrics.classification_report(y_test, y_pred_test_pagerank))
print("Confusion Matrix:")
print(metrics.confusion_matrix(y_test, y_pred_test_pagerank))
print("Accuracy:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred_test_pagerank))

             precision    recall  f1-score   support

          0       0.49      0.51      0.50     18884
          1       0.51      0.49      0.50     19713

avg / total       0.50      0.50      0.50     38597

Confusion Matrix:
[[ 9603  9281]
 [10003  9710]]
Accuracy: 0.500375676866


Stil we have very bad accuracy.
We now try tuning the pagerank algorithm using different $\alpha$ values.

In [44]:
scores = []
for alpha in np.arange(0.1, 0.9, 0.01):
    pr = nx.pagerank(g_test, alpha=0.9)
    y_pred_test_pagerank = predict_initial_problem(initial_ds_test, pr)
    scores += [metrics.accuracy_score(y_true=y_test, y_pred=y_pred_test_pagerank)]
max(scores)

0.50037567686607765

Therefore, the maximum accuracy we obtain from pagerank importance of the node is still 50% which could have been chance.

We conclude that with this dataset is difficult to rank the importance of each node and we need to look even further

## Current research on the dataset

In fact there is some relevant research on the subject from Baluja, S (2016) where a more complicated method than PageRank is applied, called Adsorption and gets a score of 62%.

## Conclusion
To sum up, we tried to invent a score for the videos that can predict which one is funnier for the youtube comedy slam dataset so that we can create a more general model. We tried first a naïve scoring model, graph clustering, pagerank and then referred to state-of-the-art research on the subject. Additionally, we examined properties of the video funniness graph and showed that it lacks many desirable properties. Therefore, it is not a promising dataset to continue examining it

### References

`Baluja, S. (2016). A Simple and Efficient Method to Handle Sparse Preference Data Using Domination Graphs: An Application to YouTube. Procedia Computer Science, 80, 2302-2311.`